In [ ]:
from google.colab import files, pathlib
up = files.upload()


import os, shutil
os.makedirs("models", exist_ok=True)
for fname in up.keys():
    shutil.move(fname, f"models/{fname}")

# check
import glob
print("Found models:", glob.glob("models/*.pkl"))


In [ ]:
# FastAPI
import os, shutil, textwrap

PROJECT = "tweet_moderation_api"
if os.path.exists(PROJECT):
    shutil.rmtree(PROJECT)
os.makedirs(f"{PROJECT}/app/static", exist_ok=True)
os.makedirs(f"{PROJECT}/models", exist_ok=True)
os.makedirs(f"{PROJECT}/tests", exist_ok=True)

for f in ["best_model_logreg_pipeline.pkl", "multi_model_pipeline.pkl"]:
    src = f"models/{f}"
    assert os.path.exists(src), f"{src} not found"
    shutil.copy(src, f"{PROJECT}/models/{f}")

utils_code = """
import re
from collections import Counter
import numpy as np

LEX_BAD = {
    "fuck":1.5,"fucking":1.5,"shit":1.0,"bitch":1.3,"slut":1.5,
    "idiot":0.8,"stupid":0.7,"moron":0.7,"dumb":0.6,
    "kill":2.0,"die":1.5,"murder":2.2,"hang":2.0,
    "creep":0.7,"pervert":1.0
}

IDENTITY_GROUPS = {
    "black":"Racist language","blacks":"Racist language",
    "asian":"Racist language","asians":"Racist language",
    "jew":"Antisemitic language","jews":"Antisemitic language",
    "muslim":"Islamophobic language","muslims":"Islamophobic language",
    "gay":"Homophobic language","gays":"Homophobic language",
    "lesbian":"Homophobic language","lesbians":"Homophobic language",
}

BASE_TAGS = {
    "toxic":"Abusive / toxic language",
    "severe_toxic":"Severe abuse",
    "obscene":"Obscene language",
    "threat":"Violence / threats",
    "insult":"Harassment / insult",
    "identity_hate":"Hate speech / identity-based"
}

def normalize(t:str)->str:
    t=t.lower()
    t=re.sub(r"https?://\\S+|www\\.\\S+"," ",t)
    t=re.sub(r"[^a-z\\s]"," ",t)
    t=re.sub(r"\\s+"," ",t).strip()
    return t

def detect_identity_subtags(toks):
    return sorted({IDENTITY_GROUPS[w] for w in toks if w in IDENTITY_GROUPS})

def foulness_meter_0_10(text_norm:str,probs:np.ndarray,labels,thresholds):
    base=6.0*float(np.max(probs))
    toks=re.findall(r"\\b[a-z]+\\b",text_norm)
    weights=[LEX_BAD[w] for w in toks if w in LEX_BAD]
    lex=min(sum(weights),2.5)
    cnt=Counter([w for w in toks if w in LEX_BAD])
    rep_boost=min(0.5*(max(cnt.values())-1),1.0) if cnt else 0
    num_on=sum(float(probs[i])>=thresholds[labels[i]] for i in range(len(labels)))
    density=min(0.4*num_on,1.0)
    return int(round(max(0,min(10,base+lex+rep_boost+density))))
"""
open(f"{PROJECT}/app/utils.py","w").write(utils_code)


main_code = """
from fastapi import FastAPI
from fastapi.responses import FileResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
import joblib,re,numpy as np
from app.utils import normalize,foulness_meter_0_10,BASE_TAGS,detect_identity_subtags,IDENTITY_GROUPS

BIN_ART=joblib.load("models/best_model_logreg_pipeline.pkl")
BIN_PIPE=BIN_ART["pipeline"]
BIN_THR=float(BIN_ART["threshold"])

ML_ART=joblib.load("models/multi_model_pipeline.pkl")
ML_PIPE=ML_ART["pipeline"]
LABELS=list(ML_ART["labels"])
THR={k:float(v) for k,v in ML_ART["thresholds"].items()}

app=FastAPI(title="Tweet Moderation API",version="2.0")
app.mount("/static",StaticFiles(directory="app/static"),name="static")

class Input(BaseModel):
    text:str

@app.get("/")
def root(): return FileResponse("app/static/index.html")

@app.get("/health")
def health(): return {"status":"ok"}

@app.post("/predict")
def predict(i:Input):
    norm=normalize(i.text)
    p_bin=float(BIN_PIPE.predict_proba([norm])[0,1])
    bin_label=int(p_bin>=BIN_THR)
    P=ML_PIPE.predict_proba([norm])[0]
    subs={lab:{"prob":float(P[k]),"label":int(P[k]>=THR[lab])} for k,lab in enumerate(LABELS)}
    tags=[BASE_TAGS[l] for l in LABELS if subs[l]["label"]]
    toks=re.findall(r"\\b[a-z]+\\b",norm)
    if subs.get("identity_hate",{}).get("label",0)==1 or any(t in IDENTITY_GROUPS for t in toks):
        tags+=detect_identity_subtags(toks)
        if "Hate speech / identity-based" not in tags: tags.append("Hate speech / identity-based")
    meter=foulness_meter_0_10(norm,np.array(P),LABELS,THR)
    return {"binary":{"prob":p_bin,"threshold":BIN_THR,"label":bin_label},"subtypes":subs,"tags":tags,"foulness_meter":meter}
"""
open(f"{PROJECT}/app/main.py","w").write(main_code)

index_html = """
<!doctype html><html><head><meta charset='utf-8'/>
<title>Tweet Moderation Demo</title>
<style>body{font-family:sans-serif;margin:24px;max-width:800px}.pill{display:inline-block;background:#eee;padding:6px 10px;border-radius:999px;margin:3px}</style>
</head><body><h1>Tweet Moderation Demo</h1>
<textarea id='txt' rows=5 style='width:100%'></textarea><br>
<button onclick='go()'>Analyze</button><span id='status'></span>
<div><b>Foulness meter:</b> <span id='meter'></span>/10</div>
<div id='tags'></div><pre id='json'></pre>
<script>
async function go(){
 let res=await fetch('/predict',{method:'POST',headers:{'Content-Type':'application/json'},body:JSON.stringify({text:txt.value})});
 let js=await res.json();
 meter.textContent=js.foulness_meter;
 tags.innerHTML=(js.tags||[]).map(t=>`<span class='pill'>${t}</span>`).join('');
 json.textContent=JSON.stringify(js,null,2);
}
</script></body></html>
"""
open(f"{PROJECT}/app/static/index.html","w").write(index_html)

open(f"{PROJECT}/requirements.txt","w").write("""fastapi==0.115.0
uvicorn[standard]==0.30.6
scikit-learn==1.4.2
pandas==2.2.2
numpy==1.26.4
joblib==1.4.2
pytest==8.3.3
""")

open(f"{PROJECT}/tests/test_api.py","w").write("""
from fastapi.testclient import TestClient
from app.main import app
c=TestClient(app)

def test_root():
    assert c.get('/').status_code==200

def test_predict():
    r=c.post('/predict',json={'text':'you idiot'})
    assert r.status_code==200
    js=r.json()
    assert 'foulness_meter' in js and 0 <= js['foulness_meter'] <= 10
""")

open(f"{PROJECT}/Dockerfile","w").write("""FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY app app
COPY models models
EXPOSE 8000
CMD ["uvicorn","app.main:app","--host","0.0.0.0","--port","8000"]
""")

open(f"{PROJECT}/README.md","w").write("""
# Tweet Moderation API

FastAPI app to classify tweets as foul or proper, with multi-label subtypes, tags, and a 0–10 foulness meter.

## Run locally
pip install -r requirements.txt
uvicorn app.main:app --reload --port 8000

## Docker
docker build -t tweet-moderation .
docker run -p 8000:8000 tweet-moderation
""")

open(f"{PROJECT}/app/__init__.py","w").write("")

print("Project generated successfully:", PROJECT)


In [ ]:
PROJECT = "tweet_moderation_api"

!pip install -q --no-cache-dir -r {PROJECT}/requirements.txt

%cd {PROJECT}
!pytest -q
%cd ..


In [ ]:
PROJECT = "tweet_moderation_api"

!touch {PROJECT}/app/__init__.py

conftest = """
import os, sys
# Add project root (one level up from tests/) to sys.path so 'app' is importable
ROOT = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)
"""
with open(f"{PROJECT}/tests/conftest.py","w") as f:
    f.write(conftest)

%cd {PROJECT}
!pytest -q
%cd ..


In [ ]:
import os, subprocess, sys
from getpass import getpass

PROJECT = "tweet_moderation_api"
REPO_URL = "https://github.com/BAKER-2/Tweet_Moderation_API.git"

token = getpass("Paste your GitHub Personal Access Token (hidden): ").strip()
assert token, "Empty token."

os.chdir(PROJECT)
subprocess.run(["git","init"], check=True)
subprocess.run(["git","config","user.name","BAKER-2"], check=True)
subprocess.run(["git","config","user.email","baker.husein01@gmail.com"], check=True)
subprocess.run(["git","branch","-M","main"], check=True)
subprocess.run(["git","add","-A"], check=True)
subprocess.run(["git","commit","-m","API: FastAPI + models + tests + Docker"], check=False)

safe_remote = f"https://{token}@github.com/BAKER-2/Tweet_Moderation_API.git"
subprocess.run(["git","remote","remove","origin"], check=False)
subprocess.run(["git","remote","add","origin", safe_remote], check=True)

# guthub push
subprocess.run(["git","push","-u","origin","main","--force"], check=True)

print("\n Pushed to GitHub: https://github.com/BAKER-2/Tweet_Moderation_API")
